<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/280_EPO_AgentDef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experimentation Portfolio Orchestrator Agent

In [ ]:
"""Experimentation Portfolio Orchestrator Agent

This agent manages the full lifecycle of experiments across an organization:
- Loads and analyzes experiment portfolio data
- Generates recommendations (scale, iterate, retire)
- Creates comprehensive portfolio reports
"""

In [ ]:
# ============================================================================
# Experimentation Portfolio Orchestrator Agent
# ============================================================================

class ExperimentationPortfolioOrchestratorState(TypedDict, total=False):
    """State for Experimentation Portfolio Orchestrator Agent"""

    # Input fields
    experiment_id: Optional[str]            # Specific experiment to analyze (None = analyze all)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                     # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]              # Execution plan (from planning_node)

    # Data Ingestion
    portfolio: List[Dict[str, Any]]         # Loaded experiment portfolio data
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "experiment_name": "AI Email Drafting for Sales",
    #   "domain": "sales",
    #   "owner": "growth_team",
    #   "status": "completed" | "running" | "planned",
    #   "start_date": "2024-10-01",
    #   "end_date": "2024-10-14" | null
    # }

    experiment_definitions: List[Dict[str, Any]]  # Loaded experiment definitions
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "hypothesis": "...",
    #   "variants": ["control", "ai_drafted"],
    #   "primary_metric": "reply_rate",
    #   "secondary_metrics": ["meeting_booked_rate"],
    #   "success_criteria": "...",
    #   "owner": "growth_team",
    #   "status": "completed"
    # }

    experiment_metrics: List[Dict[str, Any]]  # Loaded experiment metrics
    # Structure per variant:
    # {
    #   "experiment_id": "E001",
    #   "variant": "control",
    #   "reply_rate": 0.18,
    #   "meeting_booked_rate": 0.05,
    #   "sample_size": 500
    # }

    experiment_analysis: List[Dict[str, Any]]  # Loaded experiment analysis results
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "primary_metric": "reply_rate",
    #   "control_value": 0.18,
    #   "treatment_value": 0.26,
    #   "absolute_lift": 0.08,
    #   "relative_lift_percent": 44.4,
    #   "direction": "positive",
    #   "confidence": "medium",
    #   "summary": "..."
    # }

    experiment_decisions: List[Dict[str, Any]]  # Loaded experiment decisions
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "decision": "scale" | "iterate" | "retire" | "do_not_start",
    #   "rationale": "...",
    #   "recommended_action": "...",
    #   "owner": "growth_team",
    #   "decision_date": "2024-10-20"
    # }

    # Data Lookups (for fast access)
    portfolio_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> portfolio entry
    definitions_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> definition
    metrics_lookup: Dict[str, List[Dict[str, Any]]]  # experiment_id -> list of variant metrics
    analysis_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> analysis result
    decisions_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> decision

    # Portfolio Analysis
    analyzed_experiments: List[Dict[str, Any]]  # Experiments with complete analysis
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "status": "completed",
    #   "has_metrics": True,
    #   "has_analysis": True,
    #   "has_decision": True,
    #   "analysis_status": "complete" | "partial" | "missing",
    #   "needs_analysis": False,
    #   "needs_decision": False
    # }

    portfolio_summary: Dict[str, Any]  # Portfolio-level summary metrics
    # Structure:
    # {
    #   "total_experiments": 3,
    #   "completed_count": 1,
    #   "running_count": 1,
    #   "planned_count": 1,
    #   "experiments_with_analysis": 2,
    #   "experiments_with_decisions": 2,
    #   "experiments_needing_analysis": 0,
    #   "experiments_needing_decisions": 0,
    #   "domains": ["sales", "customer_support", "hr"],
    #   "total_sample_size": 1630,
    #   "average_lift_percent": 6.7
    # }

    # Experiment Analysis (calculated/updated)
    calculated_analyses: List[Dict[str, Any]]  # Newly calculated analyses
    # Same structure as experiment_analysis

    # Decision Generation
    generated_decisions: List[Dict[str, Any]]  # Newly generated decisions
    # Same structure as experiment_decisions

    # Portfolio Insights
    portfolio_insights: List[Dict[str, Any]]  # High-level insights across portfolio
    # Structure per insight:
    # {
    #   "type": "trend" | "risk" | "opportunity" | "recommendation",
    #   "title": "...",
    #   "description": "...",
    #   "experiments": ["E001", "E002"],
    #   "priority": "high" | "medium" | "low"
    # }

    # Output
    portfolio_report: str                    # Final markdown report
    report_file_path: Optional[str]          # Path to saved report file

    # Metadata
    errors: List[str]                       # Any errors encountered
    processing_time: Optional[float]        # Time taken to process


@dataclass
class ExperimentationPortfolioOrchestratorConfig:
    """Configuration for Experimentation Portfolio Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/experimentation_portfolio_reports"  # Where to save reports

    # Data file paths
    data_dir: str = "data"
    portfolio_file: str = "experiment_portfolio.json"
    definitions_file: str = "experiment_definitions.json"
    metrics_file: str = "experiment_metrics.json"
    analysis_file: str = "experiment_analysis.json"
    decisions_file: str = "experiment_decisions.json"

    # Analysis Settings
    min_sample_size: int = 100  # Minimum sample size for statistical validity
    confidence_threshold: float = 0.05  # p-value threshold for significance

    # Decision Criteria (MVP: Rule-based)
    scale_threshold_lift: float = 20.0  # Minimum % lift to recommend scaling
    iterate_threshold_lift: float = 5.0  # Minimum % lift to recommend iterating
    retire_threshold_lift: float = -10.0  # Below this % change, recommend retiring

    # Portfolio Analysis
    enable_portfolio_insights: bool = True  # Generate cross-experiment insights

    # Toolshed Integration (placeholders for future use)
    enable_progress_tracking: bool = False  # Placeholder for toolshed.progress
    enable_kpi_tracking: bool = False  # Placeholder for toolshed.kpi
    enable_hitl: bool = False  # Placeholder for toolshed.hitl




# Big Picture First (Very Important)

This code does **two main things**:

1. **Defines the agent’s “memory” (State)**
2. **Defines the agent’s “personality & rules” (Config)**

If an orchestrator agent were a person:

* **State** = the backpack it carries while working
* **Config** = the rules, preferences, and limits it follows

Everything else in the agent just **reads from and writes to these two things**.

---

## Part 1: The State — the Agent’s Backpack 🎒

```python
class ExperimentationPortfolioOrchestratorState(TypedDict, total=False):
```

This defines **everything the agent is allowed to remember while it’s running**.

Key idea:

> The agent never “thinks” in variables — it thinks in **state updates**.

Each node:

* reads some state
* adds new state
* passes the updated backpack forward

---

### 🔹 Input Fields

```python
experiment_id: Optional[str]
```

This lets the agent:

* analyze **one experiment**, or
* analyze **the whole portfolio**

➡️ This is what makes the agent flexible without changing code.

---

### 🔹 Goal & Plan

```python
goal
plan
```

Even though this is MVP and template-based, this is **huge conceptually**.

It means:

* The agent *knows what it’s trying to accomplish*
* The agent *knows the steps it plans to follow*

This is the difference between:

* a script
* and an **intent-driven system**

---

### 🔹 Data Ingestion (Raw Reality)

These fields hold **facts from the outside world**:

```python
portfolio
experiment_definitions
experiment_metrics
experiment_analysis
experiment_decisions
```

Think of these as:

* spreadsheets
* dashboards
* research notes

The agent **does not invent these** — it loads them.

This separation is critical:

> Orchestrators coordinate reality — they don’t hallucinate it.

---

### 🔹 Lookups (Speed Matters)

```python
portfolio_lookup
definitions_lookup
metrics_lookup
analysis_lookup
decisions_lookup
```

These are **indexes**.

Instead of searching through lists every time, the agent says:

> “Give me experiment E002 instantly.”

This matters because:

* orchestration systems scale
* slow lookups break pipelines
* clean architecture prevents bugs

---

### 🔹 Analyzed Experiments (Meta-Thinking)

```python
analyzed_experiments
```

This is the agent **thinking about its own work**.

It answers questions like:

* Is this experiment fully analyzed?
* Is something missing?
* Do we need a decision?

This is a **huge orchestrator concept**:

> Agents should reason about *completeness*, not just execution.

---

### 🔹 Portfolio Summary (Zoomed-Out Thinking)

```python
portfolio_summary
```

This lets the agent stop thinking about:

* single experiments

and start thinking about:

* patterns
* coverage
* gaps
* risk

This is where **portfolio intelligence** begins.

---

### 🔹 Generated Outputs

```python
calculated_analyses
generated_decisions
portfolio_insights
portfolio_report
```

These are **things the agent creates**, not loads.

Important distinction:

* loaded data = truth
* generated data = judgment

Great orchestrator agents keep those separate.

---

### 🔹 Errors & Metadata

```python
errors
processing_time
```

This makes the system:

* debuggable
* auditable
* trustworthy

Production agents *must* explain failures.

---

## Part 2: The Config — the Agent’s Rulebook 📘

```python
@dataclass
class ExperimentationPortfolioOrchestratorConfig:
```

This defines **how the agent behaves**, without changing logic.

---

### 🔹 Models & Style

```python
llm_model
temperature
```

This lets you swap models or adjust creativity **without touching nodes**.

That’s elite-level design.

---

### 🔹 Data Paths

```python
portfolio_file
definitions_file
metrics_file
...
```

Your agent:

* doesn’t care *where* data comes from
* just knows *what kind* of data it expects

This is how systems evolve cleanly.

---

### 🔹 Analysis & Decision Rules

```python
scale_threshold_lift
iterate_threshold_lift
retire_threshold_lift
```

This is incredibly important:

> Decisions are **data-driven rules**, not hardcoded opinions.

You can:

* tune behavior
* explain outcomes
* justify decisions

---

### 🔹 Future-Proofing (Toolshed Hooks)

```python
enable_progress_tracking
enable_kpi_tracking
enable_hitl
```

This shows **architectural maturity**.

You’ve designed the agent so it can:

* grow
* integrate
* specialize

without being rewritten.

---

## What You Should Focus on to Be a Top-Tier Orchestrator Developer ⭐

### 1️⃣ State design matters more than algorithms

Great agents fail because of bad state, not bad logic.

### 2️⃣ Separate facts from decisions

You did this perfectly:

* metrics ≠ analysis ≠ decisions

### 3️⃣ Make behavior configurable

Rules belong in config, not code.

### 4️⃣ Think in workflows, not functions

Every field here exists because **a node will read or write it**.

### 5️⃣ Design for auditability

Errors, decisions, timestamps, owners — this builds trust.

---

## One-Sentence Mental Model

> This code defines **what the agent knows**, **what it produces**, and **the rules it follows**, so every node can cooperate without chaos.



# Data Loading Utilities

In [ ]:
"""Data Loading Utilities

Load experiment data from JSON files and build lookup dictionaries.
"""

import json
from pathlib import Path
from typing import Dict, List, Any, Optional


def load_json_file(file_path: str) -> List[Dict[str, Any]]:
    """Load JSON file and return list of dictionaries."""
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"Data file not found: {file_path}")

    with open(path, 'r') as f:
        data = json.load(f)

    if not isinstance(data, list):
        raise ValueError(f"Expected list, got {type(data)} for {file_path}")

    return data


def load_portfolio_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment portfolio data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_definitions_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment definitions data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_metrics_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment metrics data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_analysis_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment analysis data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_decisions_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment decisions data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def build_portfolio_lookup(portfolio: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> portfolio entry."""
    return {exp["experiment_id"]: exp for exp in portfolio}


def build_definitions_lookup(definitions: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> definition."""
    return {exp["experiment_id"]: exp for exp in definitions}


def build_metrics_lookup(metrics: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """Build lookup dictionary: experiment_id -> list of variant metrics."""
    lookup: Dict[str, List[Dict[str, Any]]] = {}
    for metric in metrics:
        exp_id = metric["experiment_id"]
        if exp_id not in lookup:
            lookup[exp_id] = []
        lookup[exp_id].append(metric)
    return lookup


def build_analysis_lookup(analysis: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> analysis result."""
    return {exp["experiment_id"]: exp for exp in analysis}


def build_decisions_lookup(decisions: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> decision."""
    return {exp["experiment_id"]: exp for exp in decisions}

